# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 31 2022 12:21PM,245438,10,8466730,Beach Products Inc,Released
1,Aug 31 2022 12:17PM,245436,10,PRF-36203,Bio-PRF,Released
2,Aug 31 2022 12:17PM,245436,10,PRF-36207,Bio-PRF,Released
3,Aug 31 2022 12:17PM,245436,10,PRF-36210,Bio-PRF,Released
4,Aug 31 2022 12:17PM,245436,10,PRF-36213,Bio-PRF,Released
5,Aug 31 2022 12:14PM,245435,10,8462470,"ACI Healthcare USA, Inc.",Released
6,Aug 31 2022 12:11PM,245434,10,Enova-10565,Emerginnova,Released
7,Aug 31 2022 12:11PM,245434,10,Enova-10566,Emerginnova,Released
8,Aug 31 2022 12:11PM,245434,10,Eme-103835,Emerginnova,Released
9,Aug 31 2022 12:05PM,245433,19,ADV80005383,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
30,245433,Released,1
31,245434,Released,3
32,245435,Released,1
33,245436,Released,4
34,245438,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
245433,NaN,1.0
245434,NaN,3.0
245435,NaN,1.0
245436,NaN,4.0
245438,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
244946,20.0,46.0
245276,1.0,0.0
245281,0.0,1.0
245282,0.0,1.0
245284,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
244946,20,46
245276,1,0
245281,0,1
245282,0,1
245284,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,244946,20,46
1,245276,1,0
2,245281,0,1
3,245282,0,1
4,245284,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,244946,20,46
1,245276,1,
2,245281,,1
3,245282,,1
4,245284,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 31 2022 12:21PM,245438,10,Beach Products Inc
1,Aug 31 2022 12:17PM,245436,10,Bio-PRF
5,Aug 31 2022 12:14PM,245435,10,"ACI Healthcare USA, Inc."
6,Aug 31 2022 12:11PM,245434,10,Emerginnova
9,Aug 31 2022 12:05PM,245433,19,"AdvaGen Pharma, Ltd"
10,Aug 31 2022 11:40AM,245428,10,ISDIN Corporation
24,Aug 31 2022 11:39AM,245432,22,"NBTY Global, Inc."
25,Aug 31 2022 11:39AM,245431,22,"NBTY Global, Inc."
26,Aug 31 2022 11:39AM,245430,22,"NBTY Global, Inc."
27,Aug 31 2022 11:38AM,245429,22,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Aug 31 2022 12:21PM,245438,10,Beach Products Inc,,1
1,Aug 31 2022 12:17PM,245436,10,Bio-PRF,,4
2,Aug 31 2022 12:14PM,245435,10,"ACI Healthcare USA, Inc.",,1
3,Aug 31 2022 12:11PM,245434,10,Emerginnova,,3
4,Aug 31 2022 12:05PM,245433,19,"AdvaGen Pharma, Ltd",,1
5,Aug 31 2022 11:40AM,245428,10,ISDIN Corporation,,14
6,Aug 31 2022 11:39AM,245432,22,"NBTY Global, Inc.",,1
7,Aug 31 2022 11:39AM,245431,22,"NBTY Global, Inc.",,1
8,Aug 31 2022 11:39AM,245430,22,"NBTY Global, Inc.",,1
9,Aug 31 2022 11:38AM,245429,22,"NBTY Global, Inc.",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 31 2022 12:21PM,245438,10,Beach Products Inc,1,
1,Aug 31 2022 12:17PM,245436,10,Bio-PRF,4,
2,Aug 31 2022 12:14PM,245435,10,"ACI Healthcare USA, Inc.",1,
3,Aug 31 2022 12:11PM,245434,10,Emerginnova,3,
4,Aug 31 2022 12:05PM,245433,19,"AdvaGen Pharma, Ltd",1,
5,Aug 31 2022 11:40AM,245428,10,ISDIN Corporation,14,
6,Aug 31 2022 11:39AM,245432,22,"NBTY Global, Inc.",1,
7,Aug 31 2022 11:39AM,245431,22,"NBTY Global, Inc.",1,
8,Aug 31 2022 11:39AM,245430,22,"NBTY Global, Inc.",1,
9,Aug 31 2022 11:38AM,245429,22,"NBTY Global, Inc.",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 31 2022 12:21PM,245438,10,Beach Products Inc,1,
1,Aug 31 2022 12:17PM,245436,10,Bio-PRF,4,
2,Aug 31 2022 12:14PM,245435,10,"ACI Healthcare USA, Inc.",1,
3,Aug 31 2022 12:11PM,245434,10,Emerginnova,3,
4,Aug 31 2022 12:05PM,245433,19,"AdvaGen Pharma, Ltd",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Aug 31 2022 12:21PM,245438,10,Beach Products Inc,1.0,NaN
1,Aug 31 2022 12:17PM,245436,10,Bio-PRF,4.0,NaN
2,Aug 31 2022 12:14PM,245435,10,"ACI Healthcare USA, Inc.",1.0,NaN
3,Aug 31 2022 12:11PM,245434,10,Emerginnova,3.0,NaN
4,Aug 31 2022 12:05PM,245433,19,"AdvaGen Pharma, Ltd",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 31 2022 12:21PM,245438,10,Beach Products Inc,1.0,0.0
1,Aug 31 2022 12:17PM,245436,10,Bio-PRF,4.0,0.0
2,Aug 31 2022 12:14PM,245435,10,"ACI Healthcare USA, Inc.",1.0,0.0
3,Aug 31 2022 12:11PM,245434,10,Emerginnova,3.0,0.0
4,Aug 31 2022 12:05PM,245433,19,"AdvaGen Pharma, Ltd",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2699652,57.0,21.0
15,490365,75.0,21.0
19,490860,14.0,14.0
20,245403,18.0,4.0
21,1962654,6.0,2.0
22,1227148,5.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2699652,57.0,21.0
1,15,490365,75.0,21.0
2,19,490860,14.0,14.0
3,20,245403,18.0,4.0
4,21,1962654,6.0,2.0
5,22,1227148,5.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,57.0,21.0
1,15,75.0,21.0
2,19,14.0,14.0
3,20,18.0,4.0
4,21,6.0,2.0
5,22,5.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,57.0
1,15,Released,75.0
2,19,Released,14.0
3,20,Released,18.0
4,21,Released,6.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19,20,21,22
Status,,,,,,
Executing,21.0,21.0,14.0,4.0,2.0,0.0
Released,57.0,75.0,14.0,18.0,6.0,5.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19,20,21,22
0,Executing,21.0,21.0,14.0,4.0,2.0,0.0
1,Released,57.0,75.0,14.0,18.0,6.0,5.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19,20,21,22
0,Executing,21.0,21.0,14.0,4.0,2.0,0.0
1,Released,57.0,75.0,14.0,18.0,6.0,5.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()